In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import csv

In [2]:
!pwd

/Users/bokyung/workspace


In [9]:
# 1. set home 
home = '/Users/bokyung/'
driver = webdriver.Chrome('{}workspace/chromedriver'.format(home))
driver.implicitly_wait(3)
# 2. set year range
url = 'http://www.korea.kr/policy/mainList.do?pageIndex={}&srchRepCodeType=\
&repCodeType=&repCode=&startDate=2008-01-01&endDate=2008-11-17&srchWord='
driver.get(url.format(str(1)))

In [10]:
# 3. set pageindex range
pnum = np.arange(1,2) #test
#pnum = np.arange(1,6060) #y2008-2017
#pnum = np.arange(1,5950)#y2008-2016.11.30
#pnum = np.arange(1,1000) #전체(1) y2008-2016.11.30
#pnum = np.arange(1000,5950) #전체(2) y2008-2016.11.30
#pnum = np.arange(1,489) #전체(3) y2008.01.01-2008.11.17
#pnum = np.arange(1,47) #전체(4) y2016.12.01-2016.12.17

In [17]:
pubyear = []
pubdate = []
department = []
title= []
subtitle = []
content = []

def formatText(txt):
    dic = {'\n':' ','\t':' ','\xa0':' ','.':'. ',
           '▶':' ','◆':' ','∙':' ','❘':' ',
           ' | ':' ','|':' ','▲':' ','○':' ','□':' '}
    for x,y in dic.items():
        txt = txt.replace(x, y)
    for i in np.arange(1,10):
        txt = txt.replace('  ', ' ') 
    return txt

def get_data():
    
    for n in pnum : 
        driver.get(url.format(str(n)))

        for i in list(range(1,11)):

            try:
                driver.find_element_by_xpath('//*[@id="container"]/div/div/div[1]/div[4]/ul/li[{}]'.format(str(i))).click()
                html = driver.page_source
                soup = BeautifulSoup(html, 'lxml')

                try:
                    select = soup.select('#container > div > div.article_wrap > div.article_head > div.subject > h2')
                    title.append(select[0].text.strip()) #제목
                except:
                    title.append('')

                try:
                    select = soup.select('#container > div > div.article_wrap > div.article_head > div.ah_utill > div.subtit')
                    subtitle.append(formatText(select[0].text.strip())) #부제
                except:
                    subtitle.append('')

                try:
                    select = soup.select('#container > div > div.article_wrap > div.article_head > div.ah_utill > div.info > span')
                    department.append(select[0].text.strip()) # 관련부처
                    pubdate.append(select[1].text.strip()) # 날짜
                    pubyear.append(select[1].text.strip()[:4]) #연도
                except:
                    department.append('')
                    pubdate.append('')
                    pubyear.append('')

                try:
                    select = soup.select('#container > div > div.article_wrap > div.article_content > div.view_cont')
                    content.append(formatText(select[0].text.strip())) #내용
                except:
                    content.append('')

                driver.back()

            except:
                print(title[-1])

def export_csv():
    with open('{}/workspace/정책브리핑(전체).csv'.format(home), 'w') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['pubyear','pubdate','department','title','subtitle','content'])
        rows = zip(pubyear, pubdate, department, title, subtitle, content)
        writer.writerows(rows)    
    print('creating csv complete')

In [12]:
# start crawling
get_data()

In [19]:
len(title)

0

In [15]:
title[:5]

['19일 ‘클린사이트’ 만들기 공청회',
 '국민 3명 중 1명 ‘비만환자’…10년새 5.7%p 증가',
 '철도파업 대비 특별대책본부 운영…불편 최소화',
 '내년 국가공무원 3200명 뽑는다',
 '물류에도 RFID 적용…화물흐름 실시간 파악']

In [18]:
# export to csv 
export_csv()

creating csv complete
